## Setup


[Setup Python Virtual Environment in VSCode](/venv_setup.md)


**Security Note:** Using fixed package versions ensures stability and reproducibility, but may expose you to security vulnerabilities if not regularly updated. Balance stability and security by periodically reviewing and updating packages.

In [ ]:
# Install required packages in a virtual environment (Currently only tested with Python 3.12.4)
%pip install --upgrade pip
%pip install pandas==2.2.2
%pip install matplotlib==3.9.2
%pip install requests==2.32.3
%pip install JLC2KiCadLib==1.0.32

# Download JLCPCB Parts CSV File 

In [2]:
import requests
import os


def download_file(url, filename):
    try:
        # Check if the file already exists
        if os.path.exists(filename):
            # Delete the existing file
            os.remove(filename)
            print(f"Deleted existing file: {filename}")

        response = requests.get(f"{url}/{filename}", stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes
        with open(filename, "wb") as f:
            for chunk in response.iter_content(None):
                f.write(chunk)
        print(f"Downloaded {url}/{filename} to {filename}")
    except requests.RequestException as e:
        print(f"Download {url} failed: {e}")


# URL
url = "https://cdfer.github.io/jlcpcb-parts-database"

# Download the file
download_file(url, "jlcpcb-components-basic-preferred.csv")

Deleted existing file: jlcpcb-components-basic-preferred.csv
Downloaded https://cdfer.github.io/jlcpcb-parts-database/jlcpcb-components-basic-preferred.csv to jlcpcb-components-basic-preferred.csv


ToDo:
Bridge Rectifier Symbol
handle handmade symbols out of stock
bidirectional leds

In [30]:
import os
import json
import re
import pandas as pd

from autoLibrarySymbols import * #librarySymbols.py
from handmadeLibrarySymbols import * #handmadeLibrarySymbols.py

# missing_footprints = {}

def extract_value(description, mode, lcsc_id):
    if lcsc_id == 22818: return "16kΩ"
    if lcsc_id == 30274: return "6pF"
    if lcsc_id == 3013473: return "100nF"
    if lcsc_id == 3008298: return "4.7nF"
    
    if mode == "Resistors":
        pattern = r"(\d+(?:\.\d+)?(?:[kMGT]?)(?:Ω|ohm))"  # matches numbers followed by Ω, ohm, Ohm, or OHM
    elif mode == "Capacitors":
        pattern = r"(\d+(?:\.\d+)?(?:[pnu]?)(?:f|farad))"  # matches numbers followed by F, f, Farad, farad, pF, pf, nF, nf, uF, uf
    else:
        raise ValueError("extract_value(): Invalid mode")

    match = re.search(pattern, description, re.IGNORECASE)
    if match:
        return match.group(0)
    else:
        print(f"Error: No value found for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")
        return None

def extract_diode_type(description, pins, lcsc_id):
    if lcsc_id == 2990493: return "TVS-Bi"
    if lcsc_id == 2990473: return "TVS-Bi"
    if lcsc_id == 2990416: return "TVS-Bi"
    if lcsc_id == 2990414: return "TVS-Bi"
    if lcsc_id == 2990261: return "TVS-Bi"
    if lcsc_id == 2990124: return "TVS-Bi"
    if lcsc_id == 3019524: return "TVS-Bi"
    if lcsc_id == 1323289: return "TVS-Bi"
    if lcsc_id == 3001945: return "TVS-Uni"
    if lcsc_id == 2833277: return "TVS-Bi"
    if lcsc_id == 2975471: return "TVS-Uni"
    if lcsc_id == 78395: return "TVS-Bi"
    if lcsc_id == 2925443: return "TVS-Uni"
    if lcsc_id == 2936988: return "TVS-Bi"
    if lcsc_id == 2925441: return "TVS-Bi"
    if lcsc_id == 2925451: return "TVS-Bi"
    
    diode_types = {
        "Schottky": {"pins": 2, "type": "Schottky"},
        "Recovery": {"pins": 2, "type": "Recovery"},
        "General": {"pins": 2, "type": "General"},
        "Rectifiers": {"pins": 4, "type": "Bridge-Rectifier"},
        "Switching": {"pins": 2, "type": "Switching"},
        "Zener": {"pins": 2, "type": "Zener"},
        "Bidirectional": {"pins": 2, "type": "TVS-Bi"},
        "Unidirectional": {"pins": 2, "type": "TVS-Uni"}
    }
    for keyword, diode_info in diode_types.items():
        if keyword.casefold() in description.casefold():
            if diode_info["pins"] == pins:
                return diode_info["type"]
            else:
                print(f"Error: Number of pins ({pins}) does not match expected ({diode_info['pins']}) for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")
                return None
    print(f"Error: No diode type found for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")
    return None

def extract_transistor_type(description, pins, lcsc_id):
    if lcsc_id == 484513: return "NMOS"
    if lcsc_id == 396043: return "NMOS"
    if lcsc_id == 916398: return "NMOS"
    if lcsc_id == 296127: return "NMOS"
    if lcsc_id == 2907947: return "NPN"
    if lcsc_id == 2991999: return "PNP"
    if lcsc_id == 533008: return "NPN"    
    
    transistor_types = {
        "PNP": {"pins": 3, "type": "PNP"},
        "NPN": {"pins": 3, "type": "NPN"},
        "NChannel": {"pins": 3, "type": "NMOS"},
        "PChannel": {"pins": 3, "type": "PMOS"},
        "N-Channel": {"pins": 3, "type": "NMOS"},
        "P-Channel": {"pins": 3, "type": "PMOS"}
    }
    for keyword, transistor_info in transistor_types.items():
        if keyword.casefold() in description.casefold():
            if transistor_info["pins"] == pins:
                return transistor_info["type"]
            else:
                print(f"Error: Number of pins ({pins}) does not match expected ({transistor_info['pins']}) for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")
                return None
    print(f"Error: No transistor type found for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")
    return None

def extract_LED_value(description, lcsc_):
    if lcsc == 2985996:
        return "Red", "LED"
    elif lcsc == 34499:
        return "White", "LED"
    
    color_pattern = r"(Red|Green|Blue|Yellow|White|Emerald)"
    bi_color_pattern = r"(Red,Blue|Red,Emerald)"

    color_match = re.search(color_pattern, description, re.IGNORECASE)
    bi_color_match = re.search(bi_color_pattern, description, re.IGNORECASE)

    if bi_color_match:
        color = bi_color_match.group(0).replace("Red,Emerald", "Red/Green").replace("Red,Blue", "Red/Blue")
        return color, "LED-Bi-Colour"
    elif color_match:
        color = color_match.group(0).replace("Emerald", "Green")
        return color, "LED"
    else:
        print(f"Error: No LED value extracted for https://jlcpcb.com/partdetail/C{lcsc}  ({description})")
        return None, None

def extract_inductor_type_value(description, joints, lcsc):
    current = None
    if lcsc == 2827387: current = "300mA"
    if lcsc == 2827415: current = "900mA"
    if lcsc == 3007708: current = "410mA"
    if lcsc == 2844914: current = "305mA"
    if lcsc == 2827354: current = "5.5A"
    if lcsc == 2827458: current = "400mA"
    if lcsc == 2835403: return "120nH,80mA", "Inductor"
    
    
    # Define patterns to match inductance values
    inductance_patterns = [
        r"\b(\d+\.\d+[u|m|n]H)\b",  # e.g. 10.5uH, 10.5mH, 10.5nH
        r"\b(\d+[u|m|n]H)\b",  # e.g. 10uH, 10mH, 10nH
    ]

    # Define patterns to match current values
    current_patterns = [
        r"(\d+(\.\d+)?)A",  # e.g. 1A, 2A, 4.95A
        r"(\d+)mA",  # e.g. 100mA, 2000mA
    ]

    # Iterate over patterns and search for matches
    for pattern in inductance_patterns:
        inductance_match = re.search(pattern, description, re.IGNORECASE)
        if inductance_match:
            # Extract inductance value
            inductance = inductance_match.group(1)

            # Extract current value
            if current == None:
                for current_pattern in current_patterns:
                    current_match = re.search(current_pattern, description, re.IGNORECASE)
                    if current_match:
                        current = current_match.group(1)
                        if "mA" in current_pattern:
                            current += "mA"
                        else:
                            current += "A"
                        break
                else:
                    current = ""
                    print(f"Error: No current value extracted for https://jlcpcb.com/partdetail/C{lcsc}  ({description})")

            # Return inductance and current values
            return f"{inductance},{current}", "Inductor"
        
    if "Ferrite" in description:
        return "", "Ferrite"

    # If no match is found, print an error message and return None
    print(f"Error: No inductance value extracted for https://jlcpcb.com/partdetail/C{lcsc}  ({description})")
    return None, None

def extract_crystal_value(description, lcsc_id):
    frequency_pattern = r"(\d+(?:\.\d+)?(?:[kMGT]?)(?:Hz))"  
    capacitance_pattern = r"(\d+(?:\.\d+)?(?:[pnu]?)(?:F))"  

    frequency_match = re.search(frequency_pattern, description, re.IGNORECASE)
    capacitance_match = re.search(capacitance_pattern, description, re.IGNORECASE)

    frequency = frequency_match.group(0) if frequency_match else None
    capacitance = capacitance_match.group(0) if capacitance_match else None

    if not frequency:
        print(f"Error: No frequency found for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")
    if not capacitance:
        print(f"Error: No capacitance found for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")

    return f"{frequency},{capacitance}"

def extract_variable_resistor_type_value(description, lcsc):
    # NTC Thermistors
    if lcsc == 2991699: return "NTC", "47kΩ,4050"
    if "NTC" in description:
        pattern = r"(\d+(?:\.\d+)?Ω)"  # matches numbers followed by Ω
        match = re.search(pattern, description)
        if match:
            return "NTC", match.group(0)
        else:
            print(f"Error: Unknown resistance for https://jlcpcb.com/partdetail/C{lcsc}  ({description})")
            return None, None

    # Varistors (MOV)
    elif "Varistors" in description:
        return "MOV", ""

    # Fuses
    elif "Fuse" or "fuse" in description:
        if lcsc == 2924957: value = "1.5A"
        elif lcsc == 2838983: value = "1.5A"
        else: value = ""
        if "Resettable" in description:
            return "Fuse,Resettable", value
        else:
            return "Fuse", value

    # Unknown
    else:
        print(f"Error: Unknown type for https://jlcpcb.com/partdetail/C{lcsc}  ({description})")
        return None, None

def get_basic_or_prefered_type(df, index):
    if df.loc[index, "basic"] > 0:
        return "Basic Component"
    elif df.loc[index, "preferred"] > 0:
        return "Preferred Component"
    else:
        print("extended component found")
        return "Extended Component"

def generate_kicad_symbol_libs(symbols):
    for lib_name, symbol_list in symbols.items():
        lib_content = "(kicad_symbol_lib\n"
        lib_content += "\t(version 20231120)\n"
        lib_content += '\t(generator "CDFER")\n'
        lib_content += '\t(generator_version "8.0")\n'
        for symbol in symbol_list:
            lib_content += symbol + "\n"
        lib_content += ")\n"

        lib_content = lib_content.replace("℃", "°C")

        with open(f"JLCPCB-Kicad-Symbols/JLCPCB-{lib_name}.kicad_sym", "w") as f:
            f.write(lib_content)

df = pd.read_csv("jlcpcb-components-basic-preferred.csv")

footprints_dir = "JLCPCB-Kicad-Footprints"
footprints_lookup = {os.path.splitext(file)[0] for file in os.listdir(footprints_dir)}

symbols = {"Resistors": [], "Capacitors": [], "Diodes":[], "Transistors":[], "Inductors":[], "Variable-Resistors": [], "Crystals": []}
smt_joint_cost = 0.0017
hand_solder_joint_cost = 0.0173

componentList = []
names_lookup = []

for index in range(0, len(df)):
    # lcsc,category_id,category,subcategory,mfr,package,joints,manufacturer,basic,preferred,description,datasheet,stock,last_on_stock,price,extra
    lcsc = df.loc[index, "lcsc"]
    category = f'{df.loc[index,"category"]},{df.loc[index,"subcategory"]}'
    manufacturer = df.loc[index,"manufacturer"]
    manufacturerPartID = df.loc[index,"mfr"]
    footprint = df.loc[index, "package"]
    description = df.loc[index, "description"]
    joints = df.loc[index, "joints"]
    units = 1
    secondary_mode = ""
    subcategory = str(df.loc[index,"subcategory"])

    if "Plugin" in footprint:
        joint_cost = hand_solder_joint_cost
    else:
        joint_cost = smt_joint_cost

    price_json = json.loads(df.loc[index, "price"])
    price = float(price_json[0]["price"] + (joints * joint_cost))
    price = round(price, 3)
    price_str = f"{price}USD"
    
    if price > 3.0 or footprint == "0201" or lcsc == 882967:
        df.drop(index = index, inplace= True)
    else:
        component_class = get_basic_or_prefered_type(df, index)
        stock = df.loc[index, "stock"]
        keywords = ""
        value = None

        try:
            extra_json = json.loads(df.loc[index, "extra"])
            datasheet = extra_json["datasheet"]["pdf"]
            attributes = extra_json["attributes"]
        except:
            datasheet = df.loc[index, "datasheet"]
            attributes = []

        if df.loc[index, "category"] == "Resistors":
            value = extract_value(description, "Resistors", lcsc)
            if "x4" in footprint:
                units = 4
            lib_name = "Resistors"
            
        elif df.loc[index, "category"] == "Capacitors":
            value = extract_value(description, "Capacitors", lcsc)
            lib_name = "Capacitors"
            
        elif df.loc[index, "category"] == "Diodes" or ("TVS" in subcategory) or ("ESD" in subcategory):
            value= extract_diode_type(description, joints, lcsc)
            secondary_mode = value
            lib_name = "Diodes"
            
        elif subcategory == "Light Emitting Diodes (LED)":
            value, secondary_mode = extract_LED_value(description,lcsc)
            lib_name = "Diodes"
            
        elif subcategory == "MOSFETs" or (subcategory == "Bipolar Transistors - BJT") or (subcategory == "Bipolar (BJT)"):
            value= extract_transistor_type(description, joints, lcsc)
            secondary_mode = value
            lib_name = "Transistors"
            
        elif subcategory == "Inductors (SMD)" or (subcategory == "Ferrite Beads") or (subcategory == "Power Inductors"):
            value, secondary_mode= extract_inductor_type_value(description, joints, lcsc)
            lib_name = "Inductors"
            
        elif subcategory == "Crystals" or subcategory == "Oscillators":
            value = extract_crystal_value(description, lcsc)
            lib_name = "Crystals"
            
        elif subcategory == "NTC Thermistors" or (subcategory == "Varistors") or (subcategory == "Fuses") or (subcategory == "Resettable Fuses"):
            secondary_mode, value= extract_variable_resistor_type_value(description, lcsc)
            lib_name = "Variable-Resistors"
            
        elif df.loc[index, "category"] == "Embedded Processors & Controllers" or (df.loc[index, "category"] == "Single Chip Microcomputer/Microcontroller"):
            if update_component_inplace(lcsc, "MCUs", price, stock) == True:
                df.drop(index = index, inplace= True)
                
        elif df.loc[index, "category"] == "Connectors" or (df.loc[index, "category"] == "Key/Switch") or (df.loc[index, "category"] == "Switches"):
            if update_component_inplace(lcsc, "Connectors_Buttons", price, stock, datasheet) == True:
                df.drop(index = index, inplace= True)
                
        elif df.loc[index, "category"] == "Power Management" or (df.loc[index, "category"] == "Power Management ICs") or (lcsc == 394180):
            if update_component_inplace(lcsc, "Power", price, stock, datasheet, description) == True:
                df.drop(index = index, inplace= True)
                
        elif df.loc[index, "category"] == "Amplifiers" or (df.loc[index, "category"] == "Operational Amplifier/Comparator") or subcategory == "Analog Switches / Multiplexers" or subcategory == "Digital Potentiometers":
            if update_component_inplace(lcsc, "Analog", price, stock, datasheet, description) == True:
                df.drop(index = index, inplace= True)
                
        elif df.loc[index, "category"] == "Memory":
            if update_component_inplace(lcsc, "Memory", price, stock, datasheet, description) == True:
                df.drop(index = index, inplace= True)
                
        elif df.loc[index, "category"] == "Communication Interface Chip" or (df.loc[index, "category"] == "Communication Interface Chip/UART/485/232") or (df.loc[index, "category"] == "Interface ICs") or (df.loc[index, "category"] == "Signal Isolation Devices"):
            # print(f"{lcsc},")
            if update_component_inplace(lcsc, "Interface", price, stock, datasheet, description) == True:
                df.drop(index = index, inplace= True)
            
        if value != None:
            df.drop(index = index, inplace= True)
            symbol = generate_kicad_symbol(
                    lib_name,
                    secondary_mode,
                    lcsc,
                    datasheet,
                    description,
                    footprint,
                    value,
                    keywords,
                    price_str,
                    component_class,
                    stock,
                    category,
                    manufacturer,
                    manufacturerPartID,
                    attributes,
                    units,
                    footprints_lookup,
                    names_lookup
                )
            symbols[lib_name].append(symbol)

df.to_csv("jlcpcb-leftover.csv", index=False)

generate_kicad_symbol_libs(symbols)

# print("\nMissing Footprints:")
# for footprint, count in sorted(missing_footprints.items(), key=lambda x: x[1], reverse=True):
#     print(f"{footprint} x{count}")

Error: Number of pins (3) does not match expected (2) for https://jlcpcb.com/partdetail/C22395518  (280A 5880W 21V 6V Bidirectional 4.5V DFN2020-3L ESD and Surge Protection (TVS/ESD) ROHS)
Error: Number of pins (4) does not match expected (2) for https://jlcpcb.com/partdetail/C22395520  (5A 75W 15V 6V Bidirectional 5V SOT-143 ESD and Surge Protection (TVS/ESD) ROHS)
Error: No diode type found for https://jlcpcb.com/partdetail/C15879  (100W 12.5V 6.2V 5V SOT-363-6 ESD and Surge Protection (TVS/ESD) ROHS)
Error: No diode type found for https://jlcpcb.com/partdetail/C32677  (19V 13.3V 12V SOT-23 ESD and Surge Protection (TVS/ESD) ROHS)
Error: No diode type found for https://jlcpcb.com/partdetail/C85364  (1A@8/20us 12V 6V 5V SOT-23-6 ESD and Surge Protection (TVS/ESD) ROHS)
Error: Number of pins (3) does not match expected (2) for https://jlcpcb.com/partdetail/C2859537  (950mV@15A 35ns 1 pair of common cathodes 30A 200V TO-247AD  Fast Recovery / High Efficiency Diodes ROHS)
Error: Number o

In [28]:
import JLC2KiCadLib
import JLC2KiCadLib.JLC2KiCadLib as JLC
import os

componentList = [
178288,
2857161,
637214,
637217
]

os.chdir("..")

class Args:
    def __init__(self):
        self.footprint_creation = True
        self.symbol_creation = True
        self.footprint_lib = "JLCPCB-Kicad-Footprints"
        self.output_dir = "JLCPCB-Kicad-Library"
        self.model_base_variable = ""
        self.model_dir = "3dModels"
        self.models = ["STEP"]
        self.skip_existing = True
        self.symbol_lib = "JLCPCB-MCUs"
        self.symbol_lib_dir = "JLCPCB-Kicad-Symbols"


args = Args()

for component in componentList:
    JLC.add_component(f"C{component}", args)

os.chdir("JLCPCB-Kicad-Library")